 
<h1 align="center"> CNN FOR BEGINNERS </h1>



### Hello, I'm Prathish and an absolute beginner at CNN. My aim of this project is to learn to use CNN properly and to serve this notebook as a guide to my fellow beginners. Let's Start.

In [8]:
# Importing essential libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
# Reading the train and test datasets from Kaggle
mnist_train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
mnist_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
mnist_output =  pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')

In [10]:
#Shape of the train and test dataframes
print('train shape: ', mnist_train.shape)
print('test shape: ', mnist_test.shape)
mnist_train.head()

In [26]:
#This program will require Keras, so to check if you have installed Keras or the version of it, run this snippet

import keras
keras.__version__

#If not, you need install Keras ('pip install keras')

In [12]:
#Spliting into X and y

X = mnist_train.drop('label' , axis = 1)
y = np.array(mnist_train['label'])

In [13]:
#Perform train test split so that we can validate the model
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

In [24]:
#FLatten or normalize the values from 0-255, so that computation will be easier
X_train_flattened = X_train/255
X_test_flattened = X_test/255

In [15]:
#Reshaping the dataframe from 1x784 to 28x28
X_train_flattened = np.array(X_train_flattened).reshape(-1,28,28,1)
X_test_flattened = np.array(X_test_flattened).reshape(-1,28,28,1)

#To know why -1 in reshape, check this out https://stackoverflow.com/questions/18691084/what-does-1-mean-in-numpy-reshape

# And 1 because it has single layer. If it is RGB then replace with 3

In [16]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.imshow(X_train_flattened[2][:])

In [17]:
#Showing the grayscale image
plt.imshow(X_train_flattened[147][:], cmap='gray_r')

# Model Buliding 

In [18]:
#Import necessary modules
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization, MaxPool2D, Dropout

In [19]:
#We will use sequential model, which is feed forward

model = tf.keras.models.Sequential()

#We will use built-in flatten which will convert the 28x28 into a single one dimentional array

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation = tf.nn.relu))
model.add(keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(256, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(256, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation = tf.nn.softmax))

#We will now compile the model specifing the loss, optimizer and metric
model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

In [20]:
# Now we will fit the model

model.fit(X_train_flattened, y_train, epochs = 10)

In [21]:
#The 98% looks attractive, but the neural nets are great at fitting, the question is do they overfit?
#To answer that question, let us check the validation set

val_loss, val_acc = model.evaluate(X_test_flattened, y_test)
print(val_acc, val_loss)

In [22]:
# Seems like the model is doing well as the val_acc is close to the trained accuracy
# Summary of the model.
model.summary()

In [27]:
# Now let us predict the test csv which we need to submit

X_sub_flattened = mnist_test/255
X_sub_flattened = np.array(X_sub_flattened).reshape(-1,28,28,1)

mnist_sub = model.predict(X_sub_flattened)
predicted = mnist_sub.argmax(axis=1)

In [29]:
# Checking the type of the prediction
print(type(predicted))
print(predicted.shape)
predicted[0]

In [30]:
# For the submission format we need to have a df with 2 columns (ImageId and Label), so we create ImageId and join with the label

submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),pd.Series(predicted, name ='Label')],axis = 1)
submission.info()

#As you can see the 'Label' column is in float64 but in submission, we need it be be in int

In [32]:
# Export it as .csv
submission.to_csv('submission.csv', index=False)

## Now just submit the submission.csv. I hope the readers understood the notebook. If you have any doubts please comment and I will try to explain.

## Do not forget to upvote.

### By Prathish Murugan